In [ ]:
import os
import pandas as pd
from matplotlib import cm
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
mpl.rcParams['pdf.fonttype'] = 42
import operator

from allensdk.core.mouse_connectivity_cache import MouseConnectivityCache

In [ ]:
path = r'C:\Users\jenniferwh\AppData\Local\Allen Institute\Brain Explorer 2\Atlases\Allen Mouse Brain Common Coordinate Framework'

In [ ]:
dat = pd.read_csv(os.path.join(path, 'ontology_v2.csv'))

In [ ]:
dat.head()

In [ ]:
unionize_dat = pd.read_csv(r'C:\Users\jenniferwh\Dropbox (Personal)\python_code\jenniferwh\plaque_map\plaque_densities_per_structure.csv')
unionize_dat = unionize_dat[unionize_dat['control'] == False]
mcc = MouseConnectivityCache(manifest_file = '../connectivity/mouse_connectivity_manifest.json')
st = mcc.get_structure_tree()
ia_map = st.get_id_acronym_map()
unionize_dat['structure_id'] = [ia_map[structure] for structure in unionize_dat['structure_acronym']]

In [ ]:
unionize_dat.head()

In [ ]:
def get_mean_value_per_structure(group, age, structure_ids):
    means = []
    if len(unionize_dat[(unionize_dat['mouse_line'] == group) & (unionize_dat['age_group'] == age)]) > 0:
        isids = unionize_dat[(unionize_dat['mouse_line'] == group) & 
                             (unionize_dat['age_group'] == age)]['image_series_id'].values
        for structure_id in structure_ids:
            density = np.mean(unionize_dat[(unionize_dat['structure_id'] == structure_id) & 
                                  (unionize_dat['image_series_id'].isin(isids))]['plaque_density'])
            means.append(density*100) #To convert from fraction to percent           
        structuredat = dict(zip(structure_ids, means))
    else:
        structuredat = dict(zip(structure_ids, np.zeros(len(structure_ids))))
        isids = []
    return structuredat, len(isids)

def get_cmap(group, age, colormap=cm.gray, scale=1):
    structure_vals, n = get_mean_value_per_structure(group, age, unionize_dat['structure_id'].unique())
    rgb_vals = structure_vals.copy()
    for key in structure_vals:
        rgb_vals[key] = tuple([255*i for i in colormap(structure_vals[key]*scale)[:3]])
        rgb_vals[0] = (0, 0, 0)
    return rgb_vals, n

In [ ]:
mouse_line = 'hAPP-J20'
age = '13 mo'
scale = 10

In [ ]:
mldat.head()

In [ ]:
mldat = unionize_dat[(unionize_dat['mouse_line'] == mouse_line) &
                    (unionize_dat['age_group'] == age)]
rgb_vals, n = get_cmap(mouse_line, age, cm.hot, scale)
for key in rgb_vals.keys():
    dat.loc[dat['database_id'] == key, 'red'] = int(rgb_vals[key][0])
    dat.loc[dat['database_id'] == key, 'green'] = int(rgb_vals[key][1])
    dat.loc[dat['database_id'] == key, 'blue'] =int(rgb_vals[key][2])
dat.loc[dat['abbreviation'] == 'root', 'parent'] = 0.
dat['parent'] = [int(value) for value in dat['parent']]

In [ ]:
structure_vals, n = get_mean_value_per_structure(mouse_line, age, mldat['structure_id'].unique())
maxval = max(structure_vals.iteritems(), key=operator.itemgetter(1))[1]
maxval

In [ ]:
cbar = fig.colorbar( plt.imshow(image[i], cmap = cm.hot), fraction=0.046)
cbar.set_label('False Positive % Volume', color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')
cbar.ax.set_yticklabels([0, 
                         np.round(maxval*.125, 2), 
                         np.round(maxval*.25, 2), 
                         np.round(maxval*.375, 2),
                         np.round(maxval*.5, 2), 
                         np.round(maxval*.625, 2),
                         np.round(maxval*.75, 2),
                         np.round(maxval*.875, 2),
                         np.round(maxval, 2)])

In [ ]:
pltdat = np.array([[0, 0], [maxval, maxval]])

In [ ]:
pltdat

In [ ]:
fig = plt.figure(figsize=(2, 3))
f = plt.imshow(pltdat, cmap=cm.hot)
plt.axis('off')
cbar = fig.colorbar(f, orientation='horizontal')
cbar.set_label('False Positive % Volume', color = 'w')
plt.setp(plt.getp(cbar.ax.axes, 'yticklabels'), color='w')
cbar.ax.set_yticklabels([0, 
                         np.round(maxval*.125, 2), 
                         np.round(maxval*.375, 2),
                         np.round(maxval*.625, 2),
                         np.round(maxval*.75, 2),
                         np.round(maxval*.875, 2),
                         np.round(maxval, 2)])
plt.savefig(r'C:\Users\jenniferwh\Dropbox (Allen Institute)\Mesoscale Connectome Papers in Progress\2018 Plaque\movie\colorbars\{0}_{1}_colorbar.pdf'.format(mouse_line, age),
           bbox_inches='tight', pad_inches=0.3, format='pdf', transparent = True, dpi=1000)

In [ ]:
mapper.to_rgba(5)

In [ ]:
max(moddat['Relative Plaque Density'])

In [ ]:
dat.to_csv(os.path.join(path, 'ontology_v2.csv'), index=False)